# Crop prediction Model

Imports neccessary 

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
import seaborn as  sb 
sb.set()

Dataset exploration

In [2]:
crop_data = pd.read_csv('crop_dataset.csv')
crop_data.head()

,Crop,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,Cassava,150,50,100,5.5,1.5,60
1,Cassava,150,50,100,5.5,1.5,62
2,Cassava,150,50,100,5.5,1.5,64
3,Cassava,150,50,100,5.5,1.5,66
4,Cassava,150,50,100,5.5,1.5,68


In [3]:
crops = crop_data['Crop'].unique()
crops

array(['Cassava', 'Vanilla', 'Coffee', 'Cotton ', 'Tea', 'Tobacco',
       'Groundnuts', 'Yams', 'Maize (corn)', 'Beans', 'Irish Potato',
       'Matooke', 'Sweet Banana', 'Sugarcane'], dtype=object)

In [4]:
crop_data.shape

(37581648, 7)

In [5]:
crop_data.dropna()

,Crop,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,Cassava,150,50,100,5.5,1.5,60
1,Cassava,150,50,100,5.5,1.5,62
2,Cassava,150,50,100,5.5,1.5,64
3,Cassava,150,50,100,5.5,1.5,66
4,Cassava,150,50,100,5.5,1.5,68
...,...,...,...,...,...,...,...
37581643,Sugarcane,200,80,180,6.5,2.5,72
37581644,Sugarcane,200,80,180,6.5,2.5,74
37581645,Sugarcane,200,80,180,6.5,2.5,76
37581646,Sugarcane,200,80,180,6.5,2.5,78


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
target = label_encoder.fit_transform(crop_data.iloc[:,0])
target

array([1, 1, 1, ..., 8, 8, 8])

In [7]:
target_name = crop_data.iloc[:,0]
data = {
        'target_name':target_name,
        'target_label_code': target,
        }

target_name_target_label_tabel = pd.DataFrame(data)

In [8]:
unique_target_name = target_name_target_label_tabel['target_name'].unique()

unique_target_label_code =target_name_target_label_tabel['target_label_code'].unique() 

unique_data_dict = {
                    'name':unique_target_name,
                    'label':unique_target_label_code,
                    }

unique_value_table = pd.DataFrame(unique_data_dict)
unique_value_table

,name,label
0,Cassava,1
1,Vanilla,12
2,Coffee,2
3,Cotton,3
4,Tea,10
5,Tobacco,11
6,Groundnuts,4
7,Yams,13
8,Maize (corn),6
9,Beans,0


In [9]:
crop_data.shape

(37581648, 7)

In [10]:
un_scaled_inputs = crop_data.iloc[:,1:]
un_scaled_inputs

,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,150,50,100,5.5,1.5,60
1,150,50,100,5.5,1.5,62
2,150,50,100,5.5,1.5,64
3,150,50,100,5.5,1.5,66
4,150,50,100,5.5,1.5,68
...,...,...,...,...,...,...
37581643,200,80,180,6.5,2.5,72
37581644,200,80,180,6.5,2.5,74
37581645,200,80,180,6.5,2.5,76
37581646,200,80,180,6.5,2.5,78


scaling inputs 

In [11]:
scaler = StandardScaler()
scaled_inputs = scaler.fit_transform(un_scaled_inputs)
scaled_inputs

array([[-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.36091184],
       [-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.24596734],
       [-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.13102284],
       ...,
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.55864417],
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.67358867],
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.78853317]])

In [12]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = target[shuffled_indices]
shuffled_targets

array([ 4,  3,  3, ...,  7,  3, 11])

In [13]:
sample_count = shuffled_inputs.shape[0]

training_count, validation_count,test_count = int(0.8*sample_count),int(0.1*sample_count),int(0.1*sample_count)
print(training_count,validation_count,test_count)

30065318 3758164 3758164


spliting the data

In [14]:
train_inputs,train_targets = shuffled_inputs[:training_count],shuffled_targets[:training_count]
validation_input,validation_targets = shuffled_inputs[training_count:training_count+validation_count],shuffled_targets[training_count:training_count+validation_count]
test_inputs,test_targets = shuffled_inputs[training_count+validation_count:],shuffled_targets[training_count+validation_count:]

In [15]:
np.savez('Train_data',inputs =train_inputs,targets = train_targets)
np.savez('validation_data',inputs =validation_input,targets = validation_targets)
np.savez('test_data',inputs =test_inputs,targets = test_targets)

loading the saved data

In [16]:
trainData = np.load('Train_data.npz',allow_pickle=True)
train_inputs,train_targets = trainData['inputs'].astype(np.float64),trainData['targets']
validation_data = np.load('validation_data.npz',allow_pickle=True)
validation_inputs,validation_targets = validation_data['inputs'].astype(np.float64),validation_data['targets']
test_data = np.load('test_data.npz',allow_pickle=True)
test_inputs,test_targets = test_data['inputs'].astype(np.float64),test_data['targets']

In [17]:
print(train_targets)
original_classes = label_encoder.inverse_transform(train_targets)
print(original_classes)

[ 4  3  3 ... 10  4  4]
['Groundnuts' 'Cotton ' 'Cotton ' ... 'Tea' 'Groundnuts' 'Groundnuts']


## MODEL 

In [18]:
input_size = 6 
output_size = 14
hidden_layer_size = 100

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size,input_shape =[input_size] ,activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(output_size, activation = 'softmax'),
                            ])

model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

batch_size = 100
num_epoches = 100
early_stp = tf.keras.callbacks.EarlyStopping(patience =2)

model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = num_epoches,
         callbacks = [early_stp],
         validation_data = (validation_inputs,validation_targets),
         verbose = 1
         )

Epoch 1/100
300654/300654 [==============================] - 1780s 5ms/step - loss: 0.2966 - accuracy: 0.8089 - val_loss: 0.2846 - val_accuracy: 0.8118
Epoch 2/100
300654/300654 [==============================] - 1655s 6ms/step - loss: 0.2905 - accuracy: 0.8120 - val_loss: 0.2832 - val_accuracy: 0.8132
Epoch 3/100
300654/300654 [==============================] - 1687s 6ms/step - loss: 0.3097 - accuracy: 0.8112 - val_loss: 0.2952 - val_accuracy: 0.8109
Epoch 4/100
300654/300654 [==============================] - 1711s 6ms/step - loss: 0.3398 - accuracy: 0.8070 - val_loss: 0.2956 - val_accuracy: 0.8116


In [19]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

117443/117443 [==============================] - 285s 2ms/step - loss: 0.2957 - accuracy: 0.8115


In [20]:
test_data = test_inputs[-1:,:]
predictionns = model.predict(test_data)

1/1 [==============================] - 1s 533ms/step


In [21]:
print(test_data)

[[ 0.51075542 -0.75148209 -0.16131955 -0.48612006  0.21724451 -0.13102284]]


In [22]:
predictionns

array([[0.0000000e+00, 3.6628653e-10, 0.0000000e+00, 8.1461512e-05,
        0.0000000e+00, 0.0000000e+00, 0.0000000e+00, 1.1539477e-34,
        0.0000000e+00, 2.7410119e-28, 0.0000000e+00, 9.9991858e-01,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [61]:
predicted_class = np.argmax(predictionns)
predicted_crop = label_encoder.inverse_transform([predicted_class])
print(predicted_crop)

['Tobacco']


In [62]:
test_targets

array([ 1,  9, 13, ...,  7,  3, 11])

### prediction with percentages

In [64]:
determinants = ['N','P','K','pH','EC','moisture']
det_values = [[],]

for determinant in determinants:
        value = float(input(f'Please enter the value of {determinant}'))
        det_values[0].append(value)

scaled_query = scaler.transform(det_values)
print(scaled_query)
print(det_values)

prediction = model.predict(scaled_query)
sorted_indices = np.argsort(prediction)[0][::-1]
# prediction
predicted_crops = []
for x in sorted_indices:
    if x < len(label_encoder.classes_):
        predicted_crop = label_encoder.inverse_transform([x])[0]
        accuracy_percentage = prediction[0][x]*100
        
        if accuracy_percentage > 0.0:
            predicted_crops.append(f"{predicted_crop}:{accuracy_percentage}%")
            
for crop in predicted_crops:
    print(crop)

Please enter the value of N 120
Please enter the value of P 50
Please enter the value of K 125
Please enter the value of pH 7.
Please enter the value of EC 1.2
Please enter the value of moisture 55


[[-1.45020503 -0.6124411  -0.31877166  1.79580334 -1.34680986 -0.64827309]]
[[120.0, 50.0, 125.0, 7.0, 1.2, 55.0]]
1/1 [==============================] - ETA: 0s

H:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 79ms/step
Beans:100.0%


## Saving model 

In [65]:
import pickle 


with open ('model.pkl','wb') as f:
    pickle.dump(model, f)

INFO:tensorflow:Assets written to: ram://df77e3b0-0f07-4f9c-bfbe-3e22048a7875/assets


saving the scaler

In [28]:
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

label encoder pkl


In [29]:
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

## API code 

In [1]:
from flask import Flask, request, jsonify
import numpy as np

app = Flask(__name__)

# Load the model and scaler
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)
    
with open('label_encoder.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

determinants = ['N', 'P', 'K', 'pH', 'EC', 'moisture']

@app.route('/predict', methods=['POST'])
def predict():
    det_values = [[]]

    # Get input values from the user
    for determinant in determinants:
        value = float(request.json[determinant])
        det_values[0].append(value)

    # Scale the input values
    scaled_query = scaler.transform(det_values)

    # Perform predictions using the model
    prediction = model.predict(scaled_query)
    predicted_class = np.argmax(prediction)
    predicted_crop = label_encoder.inverse_transform([predicted_class])

    # Return the predicted crop as the output
    return jsonify({'predicted_crop': predicted_crop})

if __name__ == '__main__':
    app.run()


## Flutter code 

In [ ]:
import 'dart:convert';
import 'package:flutter/material.dart';
import 'package:http/http.dart' as http;

void main() => runApp(MyApp());

class MyApp extends StatelessWidget {
  @override
  Widget build(BuildContext context) {
    return MaterialApp(
      title: 'Crop Prediction App',
      theme: ThemeData(
        primarySwatch: Colors.blue,
      ),
      home: HomePage(),
    );
  }
}

class HomePage extends StatefulWidget {
  @override
  _HomePageState createState() => _HomePageState();
}

class _HomePageState extends State<HomePage> {
  final _formKey = GlobalKey<FormState>();
  final determinants = ['N', 'P', 'K', 'pH', 'EC', 'moisture'];
  final determinantValues = <String, double>{};

  @override
  Widget build(BuildContext context) {
    return Scaffold(
      appBar: AppBar(
        title: Text('Crop Prediction'),
      ),
      body: Padding(
        padding: EdgeInsets.all(16.0),
        child: Form(
          key: _formKey,
          child: Column(
            crossAxisAlignment: CrossAxisAlignment.start,
            children: determinants.map((determinant) {
              return TextFormField(
                decoration: InputDecoration(
                  labelText: determinant,
                ),
                validator: (value) {
                  if (value.isEmpty) {
                    return 'Please enter a value';
                  }
                  return null;
                },
                onSaved: (value) {
                  determinantValues[determinant] = double.parse(value);
                },
              );
            }).toList(),
          ),
        ),
      ),
      floatingActionButton: FloatingActionButton(
        onPressed: () {
          if (_formKey.currentState.validate()) {
            _formKey.currentState.save();
            makePredictionRequest();
          }
        },
        child: Icon(Icons.check),
      ),
    );
  }

  Future<void> makePredictionRequest() async {
    // Make the request payload
    final requestPayload = <String, dynamic>{
      for (final determinant in determinants)
        determinant: determinantValues[determinant],
    };

    // Make the request
    final url = Uri.parse('http://your-api-url/predict');
    final response = await http.post(
      url,
      body: jsonEncode(requestPayload),
      headers: {'Content-Type': 'application/json'},
    );

    // Handle the response
    if (response.statusCode == 200) {
      final responseData = json.decode(response.body);
      final predictedCrop = responseData['predicted_crop'];

      // Display the predicted crop in a dialog
      showDialog(
        context: context,
        builder: (context) => AlertDialog(
          title: Text('Predicted Crop'),
          content: Text('The predicted crop is: $predictedCrop'),
          actions: [
            TextButton(
              onPressed: () => Navigator.pop(context),
              child: Text('OK'),
            ),
          ],
        ),
      );
    } else {
      print('Request failed with status: ${response.statusCode}');
    }
  }
}
